# Exercise: Titanic Dataset - Types of Data and Handling Missing Data

To build better Machine Learning models we have to understand that there's different types of data used to describe both **features** and **labels**:

- Real Valued data: Numerical data that is often continuous, such as "Age", "Salary" or "Number of Relatives".

- Categorical data: Describes data that have labels with no particular no order, such as "Sex" (Male, Female) or "Blood Type" (A, B, AB, O). They can be represented by numbers or text, but must be converted into a numerical format for processing.

- Identity data: Data that is used to uniquely identify a record, such as an "ID", "SSN" or "Name".

As we've seen, incomplete data can negatively affect a model's perfomance or even completely stop it from working, hence why it's important to identify and correct gaps in our datasets.

In this exercise we take a deeper look into the Titanic Dataset, then build and compare Machine Learning models with the original and "cleaned" data.

## Preparing data

Let's reload the Titanic Dataset and reacquaint ourselves with its data:


In [1]:
import pandas as pd

# Load data from our dataset file into a pandas dataframe
dataset = pd.read_csv('Data/titanic.csv', index_col=False, sep=",",header=0)

# Let's take a look at the data
dataset.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Taking a careful look at the columns and data we can identify the **Real Valued** features, like `Age`, `SibSp`, `Parch` and `Fare` and **Categorical** features, such as `Survived`, `Sex`, `PClass` and `Embarked`. 


We can display a brief summary of the dataypes by using panda's `info()` method:



In [2]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


We can see that several columns are stored as numerical data (those with `int64` or `float64` types), while others contain more complex data types (those with `object` as Dtype)

Notice that `dataset.info()` also shows us the number of non-null items per column, but does not indicate if a feature is Categorical or Real valued.

## Handling Missing Data

As seen in the last Unit, the Titanic Dataset is not 100% complete:

In [3]:
# Calculate the number of empty cells in each column
# and store it in a new dataframe
missing_data = dataset.isnull().sum().to_frame()

# Rename column holding the sums
missing_data = missing_data.rename(columns={0:'Empty Cells'})

# Print the results
print(missing_data)

             Empty Cells
PassengerId            0
Survived               0
Pclass                 0
Name                   0
Sex                    0
Age                  177
SibSp                  0
Parch                  0
Ticket                 0
Fare                   0
Cabin                687
Embarked               2




Some rows in the `Age`, `Cabin` and `Embarked` have "empty" cells (or cells where the value is `null`). 

There are **many** ways to address this issue, each with pros and cons.

Let's take a look at the less complicated options:

### Option 1: Delete data with missing rows

When we have a model that cannot handle missing data, the most prudent thing to do is to remove rows that have information missing.

Let's remove some data from the `Embarked` column, which only two rows with missing data.

In [4]:
# Create a "clean" dataset where we cumulatively fix missing values
# Start by removing rows ONLY where "Embarked" has no values
print(f"The original size of our dataset was", dataset.shape)
clean_dataset = dataset.drop(dataset[dataset["Embarked"].isnull()].index)
clean_dataset = clean_dataset.reindex()

# How many rows do we have now?
print("The shape for the clean dataset is", clean_dataset.shape)


The original size of our dataset was (891, 12)
The shape for the clean dataset is (889, 12)


We can see that the offending two rows are not present in our new clean dataset.

### Option 2: Replace empty values with the mean or median for that data.

In some circumstances, our model cannot handle missing values, and we also cannot afford to remove too much data. If this is the case, sometimes we can fill missing data with the average of the rest of the dataset. 

Below, we do this for the `Age` field, and since it holds **Real valued** data and we can make a reasonable estimation of the mean Age from the remaining rows, given that >80% of these are not empty:


In [5]:
import numpy as np
# Calculate the mean value for the Age column
mean_age = clean_dataset["Age"].mean()

print("The mean age is", mean_age)

# Replace empty values in "Age" with the mean calculated above
clean_dataset["Age"].fillna(mean_age, inplace=True)

# Let's see what the clean dataset looks like now
print(clean_dataset.isnull().sum().to_frame().rename(columns={0:'Empty Cells'}))

The mean age is 29.64209269662921
             Empty Cells
PassengerId            0
Survived               0
Pclass                 0
Name                   0
Sex                    0
Age                    0
SibSp                  0
Parch                  0
Ticket                 0
Fare                   0
Cabin                687
Embarked               0


As you can see above, the `Age` field has no empty cells anymore.

### Option 3: Assign a new category to unknown categorical data

The `Cabin` field is a categorical field because there's a finite number of possible options for cabins in the Titanic. Unfortunately, many records have no cabin listed.

For this exercise it makes perfect sense to create an `Unknown` category and assign it to the cases where the cabin is unknown:


In [6]:
# Assign unknow to records where "Cabin" is empty
clean_dataset["Cabin"].fillna("Unknown", inplace=True)

# Let's see what the clean dataset looks like now
print(clean_dataset.isnull().sum().to_frame().rename(columns={0:'Empty Cells'}))

# Save the clean dataset for future use
clean_dataset.to_csv("Data/Cleaned_Titanic.csv")

             Empty Cells
PassengerId            0
Survived               0
Pclass                 0
Name                   0
Sex                    0
Age                    0
SibSp                  0
Parch                  0
Ticket                 0
Fare                   0
Cabin                  0
Embarked               0


That's it! No more missing data!

We only lost two records (where `Embarked` was empty). 

That said, we had to make some approximations to fill the missing gaps for the `Age` and `Cabin` columns, and those will certainly influence the performance of any model we train on this data. In many instances, it is better to simply remove data that are missing, or to use a model that is designed to handle missing values. Exactly which is best will depend on your circumstances.

## Model performance with cleaned Data

Let's take a quick look at how cleaning data can improve a model's performance, by building a models with either the cleaned data or the original data. 

The models we will build predict whether a person would survive or perish in the sinking of the Titanic.

We can use an algorithm called "Logistic Regression" to make this kind of prediction with the dataset we have:


In [7]:
import sklearn.model_selection as model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# We will try to predict survival based on the following features (Columns)
features = ["Age", "SibSp", "Parch", "Fare"]


# First, make dataset we will test both models on. This should be 
# data that, before cleaning, did not lack any of the features
# -- Find passengers where all features are not null
eligible_rows = (~dataset[features].isnull()).sum(axis=1) == len(features)
passengers_with_full_information = dataset[eligible_rows].PassengerId

# -- Randomly select some passenger Ids
_, test_passenger_ids = model_selection.train_test_split(passengers_with_full_information, test_size=0.30, random_state=101)

# -- Extract our feature columns for these passengers,
# -- giving us our test dataset
test_rows = dataset[dataset.PassengerId.isin(test_passenger_ids)]
X_test = test_rows[features]
y_test = test_rows.Survived

# Now create a function we can use to train and test a model
def train_logistic_regression(data):
    '''
    Trains a model to predict survival on the Titanic
    This is tested against a test dataset that had no data missing 
    '''

    # Extract training data from the dataset
    # This is all passengers who are not in the test set
    data = data[~data.PassengerId.isin(test_passenger_ids)]

    # X is our feature matrix
    X = data[features]

    # y is the label vector 
    y = data.Survived

    # train the model
    model = LogisticRegression(random_state=0).fit(X, y)

    # score is the mean accuracy on the given test data and labels
    score = model.score(X, y)

    # calculate loss
    probabilities = model.predict_proba(X_test)
    loss = metrics.log_loss(y_test, probabilities)

    return score, loss


# Train a model with the clean data
clean_score, clean_loss = train_logistic_regression(clean_dataset)
print(f"Cleaned data. Score: {clean_score}, Loss: {clean_loss}")


# Train a model with the original (not cleaned) data
# Note that the model will not run if there are values missing,
# so for the sake of this exercise, we will replace missing values 
# with 0s
original_data = dataset.copy(deep=True)
original_data["Age"].fillna(0, inplace=True)
original_score, original_loss = train_logistic_regression(original_data)

print(f"Original data. Score: {original_score}, Loss: {original_loss}")

Cleaned data. Score: 0.7096296296296296, Loss: 0.6734830137281654
Original data. Score: 0.6982248520710059, Loss: 0.6805265405413654


The **clean** model yields both a better score and a smaller loss, as expected.

## Summary

In this Unit you've learned about different types of data and the importance of dealing with missing values before training our model.

Recall that we discussed the options of deleting rows with missing data, replacing `null` values in a numeric column with the `mean` of its valid values, and creating new categories for unknown categorical data, but that there are more sophisticated ways to make these corrections. 

Finally, we built and compared models built with the original dataset and a "cleaned" dataset, with the latter model showing the best results.
